In [8]:
import zlib
import bz2
import brotli
import lzma
import secrets
import string
import struct
import math
import bitarray

In [21]:
class Satellite:
    SYNC_WORD = b'\x1A\xCF\xFC\x1D'  # Example sync word

    def __init__(self, name):
        self.name = name
        self.compressor = brotli

    def generate_tmtc_frames(self, num_frames, payload_size=246):
        """Generate realistic TMTC frames with headers, payload, and CRC."""
        frames = []

        for _ in range(num_frames):
            # Header fields
            spacecraft_id = secrets.randbelow(256)  # Example: 1-byte spacecraft ID (0-255)
            frame_type = secrets.randbelow(16)      # Example: 4-bit frame type (0-15)
            header = struct.pack('>BB', spacecraft_id, frame_type)  # Pack into 2 bytes
            
            # Generate payload with cryptographic randomness
            payload = ''.join(secrets.choice(string.ascii_uppercase + string.digits) for _ in range(payload_size))
            payload_bytes = payload.encode('ascii')  # Convert to bytes
            
            # Concatenate sync word, header, and payload
            frame_data = self.SYNC_WORD + header + payload_bytes
            
            # Compute CRC (32-bit) over the frame data
            crc = zlib.crc32(frame_data) & 0xFFFFFFFF  # Ensure it’s a 32-bit value
            crc_bytes = struct.pack('>I', crc)  # Pack CRC into 4 bytes (big-endian)
            
            # Append CRC to the frame
            full_frame = frame_data + crc_bytes
            
            frames.append(full_frame)
        
        return frames

    def send_frames(self, frames):
        """Simulate sending frames."""
        total_size = sum(len(frame) for frame in frames)  # No need to encode to UTF-8, frames are binary
        print(f"{self.name}: Sending {len(frames)} TMTC frames to ground station. Total size: {total_size} bytes.")
        return frames, total_size

    def send_compressed_frames(self, frames):
        """Compress and send frames using Golomb-Rice compression."""
        # Compress the entire frame byte by byte
        compressed_frames = []

        for frame in frames:
            compressed_bits = self.compressor.compress(frame)
            compressed_frames.append(compressed_bits)

        total_compressed_size = sum(len(frame) for frame in compressed_frames)

        print(f"{self.name}: Sending {len(compressed_frames)} compressed TMTC frames to ground station. Total size: {total_compressed_size} bits.")
        return compressed_frames, total_compressed_size


class GroundStation:
    def __init__(self, name):
        self.name = name
        self.compressor = brotli

    def receive_frames(self, frames):
        """Simulate receiving frames."""
        print(f"{self.name}: Received {len(frames)} TMTC frames from satellite.")
        return frames

    def receive_compressed_frames(self, compressed_frames):
        """Receive and decompress frames using Golomb-Rice compression."""
        decompressed_frames = []

        for compressed_bits in compressed_frames:
            # Decompress the bits into byte values
            decompressed_values = self.compressor.decompress(compressed_bits)
            # Convert decompressed values back to bytes
            frame = bytearray(decompressed_values)
            decompressed_frames.append(frame)

        print(f"{self.name}: Received and decompressed {len(decompressed_frames)} TMTC frames from satellite.")
        return decompressed_frames

In [26]:
# Simulate a pass where the satellite and ground station exchange frames
def simulate_uncompressed_pass():
    satellite = Satellite("Satellite-1")
    ground_station = GroundStation("GroundStation-1")

    # Satellite generates and sends frames
    frames = satellite.generate_tmtc_frames(num_frames=1)
    sent_frames, total_size = satellite.send_frames(frames)

    # Ground station receives the frames
    ground_station.receive_frames(sent_frames)

    return total_size

# Simulate a pass where the satellite and ground station exchange compressed frames
def simulate_compressed_pass():
    satellite = Satellite("Satellite-1")
    ground_station = GroundStation("GroundStation-1")

    # Satellite generates, compresses, and sends frames
    frames = satellite.generate_tmtc_frames(num_frames=1)
    compressed_frames, total_compressed_size = satellite.send_compressed_frames(frames)

    # Ground station receives and decompresses the frames
    ground_station.receive_compressed_frames(compressed_frames)

    return total_compressed_size

In [27]:
# Simulate both passes and compare
print("\nSimulating uncompressed pass:\n")
uncompressed_size = simulate_uncompressed_pass()

print("\nSimulating compressed pass:\n")
compressed_size = simulate_compressed_pass()

# Compare sizes
print(f"\nTotal size of uncompressed frames: {uncompressed_size} bytes")
print(f"Total size of compressed frames: {compressed_size} bytes")

if compressed_size < uncompressed_size:
    savings = ((uncompressed_size - compressed_size) / uncompressed_size) * 100
    print(f"Compression reduced the size by {savings:.2f}%")
else:
    print("Compression did not reduce the size.")


Simulating uncompressed pass:

Satellite-1: Sending 1 TMTC frames to ground station. Total size: 256 bytes.
GroundStation-1: Received 1 TMTC frames from satellite.

Simulating compressed pass:

Satellite-1: Sending 1 compressed TMTC frames to ground station. Total size: 203 bits.
GroundStation-1: Received and decompressed 1 TMTC frames from satellite.

Total size of uncompressed frames: 256 bytes
Total size of compressed frames: 203 bytes
Compression reduced the size by 20.70%
